In [ ]:
# pip install requests bs4 lxml xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 2.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin

In [8]:
import csv
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin

def CleanData(InfoString):
    """Clean the data by removing unwanted characters"""
    InfoString = InfoString.replace('\n', '',)  
    InfoString = InfoString.replace('\r', '',)  
    InfoString = InfoString.replace(' ', '',)
    InfoString = InfoString.replace('年', '-', 1)  
    InfoString = InfoString.replace('月', '-', 1)
    InfoString = InfoString.replace('日', '', 1)
    return InfoString

def ExtractWeatherForCity(city):
    """Scrape weather data for the specified city and append to the same CSV file"""
    # Define the base URL for the weather history page for different cities
    url = f'http://www.tianqihoubao.com/lishi/{city}.html'
    
    # Open the same CSV file to append data from different cities
    output_file = 'Weather_Data_All_Cities.csv'
    with open(output_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # If it's the first city, write the header
        if file.tell() == 0:  # Check if file is empty, i.e., first write
            writer.writerow(['Date', 'City', 'Weather Condition', 'Temperature', 'Wind Force and Direction'])  # CSV headers
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
            'Connection': 'close'
        }
        html = requests.get(url, headers=headers)
        html.encoding = 'gb2312'  # Correctly set the encoding to match the website
        bsObj = BeautifulSoup(html.text, 'lxml')

        # Find the weather data for the city
        year_weather_data = bsObj.find_all('div', class_="box pcity")  # Weather data for all years
        year = 2010
        for quarter_data in year_weather_data:  # Iterate through quarterly data
            year += 1
            if year <= 2021:  # Restrict to data up to 2020
                quarter_list = quarter_data.find_all('ul')  # Quarterly data
                for month_data in quarter_list:  # Iterate through months in a quarter
                    month_links = month_data.find_all('a')  # Extract links for each month
                    for link in month_links:
                        month_url = urljoin('http://www.tianqihoubao.com', link['href'])
                        try:
                            month_html = requests.get(month_url, headers=headers)
                            month_html.encoding = 'gb2312'  # Correct encoding
                            month_bsObj = BeautifulSoup(month_html.text, 'lxml')
                            month_rows = month_bsObj.find_all('tr')  # Rows of monthly data
                            for i, day_row in enumerate(month_rows):
                                if i == 0:  # Skip the header row
                                    continue
                                day_data = [CleanData(td.get_text()) for td in day_row.find_all('td')]
                                if len(day_data) == 4:
                                    day_data.insert(1, city)  # Insert city name at the second column
                                    writer.writerow(day_data)  # Write data to CSV file
                        except Exception as e:  # Handle request rate limit
                            print(f"Error scraping {city} data for month: {link['href']}, retrying...")
                            time.sleep(5)
                            continue
                print(f'Finished scraping data for {year}...')
            else:
                break
    print(f'Scraping data for {city} completed!')
    return

# Example: Scrape weather data for Beijing, Shanghai, and Guangzhou
ExtractWeatherForCity('beijing')
ExtractWeatherForCity('shanghai')
ExtractWeatherForCity('lasa')
ExtractWeatherForCity('haikou')

Finished scraping data for 2011...
Finished scraping data for 2012...
Finished scraping data for 2013...
Finished scraping data for 2014...
Finished scraping data for 2015...
Finished scraping data for 2016...
Finished scraping data for 2017...
Finished scraping data for 2018...
Finished scraping data for 2019...
Finished scraping data for 2020...
Finished scraping data for 2021...
Scraping data for beijing completed!
Finished scraping data for 2011...
Finished scraping data for 2012...
Finished scraping data for 2013...
Finished scraping data for 2014...
Finished scraping data for 2015...
Finished scraping data for 2016...
Finished scraping data for 2017...
Finished scraping data for 2018...
Finished scraping data for 2019...
Finished scraping data for 2020...
Finished scraping data for 2021...
Scraping data for shanghai completed!
Finished scraping data for 2011...
Finished scraping data for 2012...
Finished scraping data for 2013...
Finished scraping data for 2014...
Finished scrapi